<a href="https://colab.research.google.com/github/vifirsanova/empi/blob/main/SCENARIOS/gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# импорт библиотек

In [ ]:
!pip install transformers
import json
import unicodedata
import re
import nltk
nltk.download('punkt')
from transformers import AutoTokenizer

# токенизатор

In [176]:
class Tokenizer():
  """
  Токенизатор:
  - нормализация кодировок по методу NFC (True, False)
  - чистка по заданному паттерну с помощью RegEx (паттерн вида r'...')
  - приведение к нижнему регистру (True, False)
  - посимвольная, пословная сегментация, n-граммная сегментация или BPE (опции: symbol (str), word (str), n (int), subword (str))
  """
  def __init__(self):
    pass

  def normalize(self, text):
    return unicodedata.normalize('NFC', text)

  def clean(self, text, pattern):
    return re.sub(pattern, '', text)

  def to_lower(self, text):
    return text.lower()

  def segmentation(self, text, setting):
    # посимвольная
    if setting == 'symbol':
      return [*text]

    # пословная
    elif setting == 'word':
      return nltk.word_tokenize(text)

    # n-граммная
    elif isinstance(setting, int):
      temp = nltk.word_tokenize(text)
      ngrams = []
      for i in range(len(temp) - setting+1):
        ngrams.append(' '.join(temp[i:i + setting]))
      return ngrams

    # подсловная
    else:
      tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
      return tokenizer.tokenize(text)

пример использования токенизатора

In [179]:
test_text = ["Этот текст содержит .#,артефа-кты",
             "Образец посимвольной токенизации",
             "Образец пословной токенизации",
             "Образец биграммной токенизации",
             "Образец триграммной токенизации",
             "Образец подсловной токенизации"]

print(test_text[0], '\n', Tokenizer().clean(text=test_text[0], pattern=r'[^\w\s]'))
print()
print(test_text[1], '\n', Tokenizer().segmentation(text=test_text[1], setting='symbol'))
print()
print(test_text[2], '\n', Tokenizer().segmentation(text=test_text[2], setting='word'))
print()
print(test_text[3], '\n', Tokenizer().segmentation(text=test_text[3], setting=2))
print()
print(test_text[4], '\n', Tokenizer().segmentation(text=test_text[4], setting=3))
print()
print(test_text[5], '\n', Tokenizer().segmentation(text=test_text[5], setting='subword'))

Этот текст содержит .#,артефа-кты 
 Этот текст содержит артефакты

Образец посимвольной токенизации 
 ['О', 'б', 'р', 'а', 'з', 'е', 'ц', ' ', 'п', 'о', 'с', 'и', 'м', 'в', 'о', 'л', 'ь', 'н', 'о', 'й', ' ', 'т', 'о', 'к', 'е', 'н', 'и', 'з', 'а', 'ц', 'и', 'и']

Образец пословной токенизации 
 ['Образец', 'пословной', 'токенизации']

Образец биграммной токенизации 
 ['Образец биграммной', 'биграммной токенизации']

Образец триграммной токенизации 
 ['Образец триграммной токенизации']

Образец подсловной токенизации 
 ['о', '##б', '##р', '##а', '##з', '##е', '##ц', 'п', '##о', '##д', '##с', '##л', '##ов', '##н', '##о', '##и', 'т', '##о', '##к', '##е', '##н', '##и', '##з', '##а', '##ц', '##ии']


# поиск по графу

In [163]:
class Search():
  """
  Поиск по графу для реализации двух сценариев
  1. Шифрование данных: поиск данных, уязвимых для хакеров, с помощью графа знаний
  - ищет указание на персональные данные пользователя с помощью поиска по базе знаний и шифрует их
  - возвращает исходный текст с зашифрованными данными
  2. Извлечение информации для генерации информативного ответа
  - ищет по графу релевантную информацию и извлекает текст
  - текст можно использовать для обусловленной генерации с помощью LLM
  """
  def __init__(self):
    self.path = set()

  def graph_search(self, data, query):
    """
    Рекурсивная функция для поиска совпадений (query) по графу (data):
    - функция проходит по трем измерениям графа
    - ищет точные совпадения или наличие поискового запроса внутри последовательности
    """
    for k in data:
      if query in k.split() or query == k:
        self.path.add(data[k]) if isinstance(data[k], dict) == False else self.path.add(str(data[k]))
      if isinstance(data[k], dict):
        self.graph_search(data=data[k], query=query)
      else:
        if isinstance(data[k], str):
          if query in data[k].split() or query == data[k]:
            self.path.add(data[k]) if isinstance(data[k], dict) == False else self.path.add(str(data[k]))
        else:
          for elem in data[k]:
            if query in elem.split() or query == elem:
              self.path.add(data[k]) if isinstance(data[k], dict) == False else self.path.add(str(data[k]))
    return self.path

пример использования поиска по графу

In [167]:
# граф знаний: открыть
with open('/content/drive/MyDrive/updated.json', 'r') as f:
  data = json.load(f)

for test_query in ['номер телефона', 'iphone', 'персональные данные']:
  print(f'Результаты графового поиска по запросу <{test_query}>')
  print(Search().graph_search(data=data, query=test_query))
  print()

Результаты графового поиска по запросу <номер телефона>
{'зашифровано'}

Результаты графового поиска по запросу <iphone>
{'технология автоматического воспроизведения текста, например, функция “прямая речь” в iphone'}

Результаты графового поиска по запросу <персональные данные>
{"{'адрес': 'зашифровано', 'e-mail': 'зашифровано', 'номер телефона': 'зашифровано', 'фамилия': 'зашифровано', 'имя': 'зашифровано', 'отчество': 'зашифровано', 'место рождения': 'зашифровано', 'год рождения': 'зашифровано', 'дата рождения': 'зашифровано', 'адрес регистрации': 'зашифровано', 'адрес проживания': 'зашифровано', 'почтовый адрес': 'зашифровано', 'адрес электронной почты': 'зашифровано', 'паспортные данные': 'зашифровано', 'серия паспорта': 'зашифровано', 'номер паспорта': 'зашифровано', 'кем и когда выдан паспорт': 'зашифровано', 'информация об образовании': 'зашифровано', 'наименование образовательного учреждения': 'зашифровано', 'инормация о месте работы': 'зашифровано', 'состав семьи': 'зашифрован

# моделька

In [203]:
class Encoder():
  """
  Класс для энкодера:
  - токенизирует промпт
  - шифрует персональные данные с помощью поиска по графу
  - ищет релевантную информацию в графе знаний
  - формирует блок
  - обновляет цепочку блоков
  - сохраняет информацию в *.json для интерпретации результатов генерации
  """
  def __init__(self, block, prompt, encoder_settings):
    self.prompt = prompt
    self.block = block
    self.tokenizer_settings, self.search_data = encoder_settings

  def _tokenizer(self):
    """
    Токенизатор:
    - нормализация кодировок
    - чистка
    - приведение к нижнему регистру (True, False)
    - пословная сегмантация или BPE
    """
    # инициализация токенизатора
    tokenizer = Tokenizer()
    # нормализация специальных символов методом NFC, если True
    self.prompt = tokenizer.normalize(self.prompt) if self.tokenizer_settings['normalize'] == True else self.prompt
    # чистка от артефактов 'save_punctuation' или от всех знаков препинания 'full_cleaning'
    pattern = r'[^\w\s\d.,!?;:()\'"\[\]{}<>-]' if self.tokenizer_settings['cleaning'] == 'save_punctuation' else r'[^\w\s\d]'
    self.prompt = tokenizer.clean(text=self.prompt, pattern=pattern)
    # приведение к нижнему регистру, если True
    self.prompt = tokenizer.to_lower() if self.tokenizer_settings['lower'] == True else self.prompt
    # сегментация: по символам 'symbol', по словам 'word', по n-граммам n (int) или подсловам 'subword'
    tokenized_prompt = tokenizer.segmentation(text=self.prompt, setting=self.tokenizer_settings['segmentation'])
    return tokenized_prompt

  def _retrieval(self, tokenized_prompt):
    """
    Извлечение из графа знаний информации для:
    1. шифрования данных
    2. поиска релевантной для пользователя информации для обусловленной генерации текста
    """
    # инициализация модуля поиска по графу
    search = Search()
    cyphered_prompt = []  # токены, отфильтрованные алгоритмом шифрования
    prompt_info = []  # релевантная информация из графа
    for token in tokenized_prompt:
      search_result = search.graph_search(data=self.search_data, query=token)
      # поиск по токенам до совпадения со специальной меткой "зашифровано"
      if 'зашифровано' in search_result:
        # меняем токен с информацией на специальный
        token = '<CYPHERED>'
      else:
        # если токен не зашифрован, проверяем, нашлась ли по нему полезная информация
        if len(search_result) > 0:
          # вносим информацию в prompt_info
          prompt_info.append(search_result)
      # обновляем список токенов
      cyphered_prompt.append(token)
    return cyphered_prompt, prompt_info

  def update_blockchain(self, current_block, cyphered_prompt, prompt_info):
    current_block = [cyphered_prompt, prompt_info]
    return current_block
  def save_block(self):
    tokenized_prompt = self._tokenizer()
    cyphered_prompt, prompt_info = self._retrieval(tokenized_prompt)
    current_block = self.update_blockchain(self.block, cyphered_prompt, prompt_info)
    user_block = current_block # to json
    return user_block

class Decoder():
  """
  Класс для декодера:
  - принимает на вход блок, сформированный энкодером
  - преобразует блок к виду вектора
  - инициализирует вектор как префикс для обусловленной генерации с LLM
  - оценивает выдачу LLM по критериям "приватность" и "информативность" на основе графа
  - оценивает выдачу LLM на основе отзывов людей
  - выводит взвешенное среднее 3-х оценок для выбора лучшего варианта генерации
  - возвращает выдачу с наибольшей финальной оценкой
  """
  def __init__(self, current_block, decoder_settings):
    self.current_block = current_block
    self.decoder_settings = decoder_settings
  def block2chain(self):
    vectorized_block = self.current_block
    return vectorized_block
  def language_model(self, vectorized_block):
    lm_outputs = [vectorized_block, vectorized_block]
    return lm_outputs
  def privacy_consistency_check(self, lm_outputs):
    privacy_scores = {}
    consistency_scores = {}
    for output in lm_outputs[0][0]:
      privacy_scores[output] = 1
      consistency_scores[output] = 1
    return privacy_scores, consistency_scores
  def human_evaluation(self, lm_outputs):
    preference_scores = {}
    for output in lm_outputs[0][0]:
      preference_scores[output] = 1
    return preference_scores
  def argmax(self, privacy_scores, consistency_scores, preference_scores):
    best_model_response = privacy_scores, consistency_scores, preference_scores
    return best_model_response
  def model_output(self):
    vectorized_block = self.block2chain()
    lm_outputs = self.language_model(vectorized_block)
    privacy_scores, consistency_scores = self.privacy_consistency_check(lm_outputs)
    preference_scores = self.human_evaluation(lm_outputs)
    best_model_response = self.argmax(privacy_scores, consistency_scores, preference_scores)
    outputs = best_model_response # text only
    return best_model_response

class Model():
  """
  Класс для модели:
  - обрабатывает блок и промпт на данной итерации
  - энкодер принимает текущий блок и промпт и возвращает блок
  - декодер принимает блок энкодера и возвращает лучшую выдачу для данного блока
  - модель возвращает строку - лучшую генерацию декодера
  """
  def __init__(self, block, prompt, settings):
    self.block = block
    self.prompt = prompt
    # кортеж: настройки энкодера и настройки декодера
    self.encoder_settings, self.decoder_settings = settings

  def answer(self):
    current_block = Encoder(self.block, self.prompt, self.encoder_settings).save_block()
    result = Decoder(current_block, self.decoder_settings).model_output()
    return result

class Config():
  """
  Конфигурация модели (кортеж):
  - настройки энкодера
  - настройки декодера
  """
  def __init__(self):
    self.tokenizer_settings = dict()
    self.search_settings = dict()

  def _tokenizer_settings(self, normalize=True, cleaning='save_punctuation', lower=True, segmentation='word'):
    """
    Настройка энкодера:
    - нормализация: True / False
    - чистка: 'save_punctuation' / 'full_cleaning' / n, где n - число n-грамм
    - приведение к нижнему регистру: True / False
    - тип сегментации: 'symbol', 'word', 'subword'
    """
    self.tokenizer_settings['normalize'] = normalize
    self.tokenizer_settings['cleaning'] = cleaning
    self.tokenizer_settings['lower'] = lower
    self.tokenizer_settings['segmentation'] = segmentation
    return self.tokenizer_settings

  def _search_data(self, search_data):
    """
    Загрузка графа знанийдля поиска
    """
    self.search_data = data
    return self.search_data

  def get_config(self, tokenizer_settings, search_data):
    encoder_settings = self.tokenizer_settings, self.search_data
    decoder_settings = None
    return encoder_settings, decoder_settings

!wget -O block.json https://raw.githubusercontent.com/vifirsanova/empi/main/SCENARIOS/%D0%90%D0%BD%D1%8F_block\(1\).json

# загружаем блок с предыдущей итерации
with open('block.json', 'r') as f:
  block = json.load(f)

# принимаем промпт пользователя
prompt = 'Расскажи мне про медитации. Хочешь узнать мой номер телефона?'

# настройки энкодера и декодера
config = Config()
# изменяем настройку токенизатора
tokenizer_settings = config._tokenizer_settings(cleaning='save_punctuation', segmentation='word', lower=False)
#print(tokenizer_settings)

# загружаем граф для поиска
with open('/content/drive/MyDrive/updated.json', 'r') as f:
  data = json.load(f)
search_data = config._search_data(search_data=data)
#print(search_data)

settings = config.get_config(tokenizer_settings, search_data)
#print(settings)
# инициализируем модель
model = Model(block, prompt, settings)

# генерируем ответ (KB IR + Blockchain + Weighted LLM Generation)
model.answer()

--2024-04-17 13:48:28--  https://raw.githubusercontent.com/vifirsanova/empi/main/SCENARIOS/%D0%90%D0%BD%D1%8F_block(1).json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1250 (1.2K) [text/plain]
Saving to: ‘block.json’

block.json          100%[===================>]   1.22K  --.-KB/s    in 0s      

2024-04-17 13:48:28 (66.2 MB/s) - ‘block.json’ saved [1250/1250]

['Расскажи', 'мне', 'про', 'медитации', '.', 'Хочешь', 'узнать', 'мой', '<CYPHERED>', '<CYPHERED>', '<CYPHERED>']
[{'вести дневник можно от руки, можно набирать тексты на компьютере, а можно воспользоваться специальными приложениями, такими как notion. я люблю писать от руки. это не только романтично, но и наделяет особенным смыслом каждое мое слово на бумаге, позволяет эффективнее сбросить накоп

({'Расскажи': 1,
  'мне': 1,
  'про': 1,
  'медитации': 1,
  '.': 1,
  'Хочешь': 1,
  'узнать': 1,
  'мой': 1,
  '<CYPHERED>': 1},
 {'Расскажи': 1,
  'мне': 1,
  'про': 1,
  'медитации': 1,
  '.': 1,
  'Хочешь': 1,
  'узнать': 1,
  'мой': 1,
  '<CYPHERED>': 1},
 {'Расскажи': 1,
  'мне': 1,
  'про': 1,
  'медитации': 1,
  '.': 1,
  'Хочешь': 1,
  'узнать': 1,
  'мой': 1,
  '<CYPHERED>': 1})

In [ ]:
# todo
# tokenizer: del stop words?